In [1]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import arviz as az

from tbdynamics.calibration.plotting import plot_spaghetti, plot_output_ranges
from tbdynamics.tools.inputs import load_targets
from tbdynamics.settings import CM_PATH, OUT_PATH, DOCS_PATH
from tbdynamics.constants import compartments
import pickle
import pandas as pd
import plotly.express as px
import numpy as np
from  tbdynamics.camau.calibration.utils import get_bcm
from tbdynamics.constants import quantiles
from estival.sampling import tools as esamp


WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# loaded_inference_data = az.from_netcdf(OUT_PATH / 'inference_data1.nc')
# idata = az.from_netcdf(OUT_PATH / 'extracted_idata.nc')
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
targets = load_targets(CM_PATH / "targets.yml")

In [3]:
idata_raw = az.from_netcdf(OUT_PATH / 'camau/calib_full_out1.nc')

In [4]:
burnt_idata = idata_raw.sel(draw=np.s_[1000:])

In [5]:
idata_extract = az.extract(burnt_idata, num_samples=200)

In [6]:
idata_extract

<xarray.Dataset> Size: 38kB
Dimensions:                                         (sample: 200)
Coordinates:
  * sample                                          (sample) object 2kB MultiIndex
  * chain                                           (sample) int32 800B 5 ... 4
  * draw                                            (sample) int32 800B 2722 ...
Data variables: (12/22)
    contact_rate                                    (sample) float64 2kB 10.5...
    rr_infection_latent                             (sample) float64 2kB 0.03...
    rr_infection_recovered                          (sample) float64 2kB 0.10...
    progression_multiplier                          (sample) float64 2kB 0.71...
    early_progression_multiplier                    (sample) float64 2kB 1.09...
    seed_time                                       (sample) float64 2kB 1.83...
    ...                                              ...
    detection_reduction                             (sample) float64 2kB 0.09...
    notif_dispersion                                (sample) float64 2kB 44.1...
    latent_dispersion                               (sample) float64 2kB 4.65...
    passive_notification_smear_positive_dispersion  (sample) float64 2kB 43.2...
    acf_detectionXact3_trail_dispersion             (sample) float64 2kB 10.0...
    acf_detectionXact3_control_dispersion           (sample) float64 2kB 24.4...
Attributes:
    created_at:                 2025-02-22T08:29:25.717447
    arviz_version:              0.17.0
    inference_library:          pymc
    inference_library_version:  5.2.0
    sampling_time:              3438.9699771404266
    tuning_steps:               5000

In [7]:
# outputs = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs.pkl', 'wb') as f:
#      pickle.dump(outputs, f)

In [ ]:
# with open(OUT_PATH /'camau/quant_outputs.pkl', 'rb') as f:
#     outputs = pickle.load(f)
scenario_config = {"detection_reduction": True, "contact_reduction": False}

    # Base scenario (calculate outputs for all indicators)
bcm = get_bcm(params, scenario_config, homo_mixing=False)
base_results = esamp.model_results_for_samples(idata_extract, bcm).results
base_quantiles = esamp.quantiles_for_results(base_results, quantiles)

In [9]:
base_quantiles

variable acf_detectionXact3_control                          \
quantile                      0.025 0.250 0.500 0.750 0.975   
time                                                          
1800.0                          0.0   0.0   0.0   0.0   0.0   
1800.1                          0.0   0.0   0.0   0.0   0.0   
1800.2                          0.0   0.0   0.0   0.0   0.0   
1800.3                          0.0   0.0   0.0   0.0   0.0   
1800.4                          0.0   0.0   0.0   0.0   0.0   
...                             ...   ...   ...   ...   ...   
2049.6                          0.0   0.0   0.0   0.0   0.0   
2049.7                          0.0   0.0   0.0   0.0   0.0   
2049.8                          0.0   0.0   0.0   0.0   0.0   
2049.9                          0.0   0.0   0.0   0.0   0.0   
2050.0                          0.0   0.0   0.0   0.0   0.0   

variable acf_detectionXact3_controlXorgan_pulmonary                          \
quantile                                      0.025 0.250 0.500 0.750 0.975   
time                                                                          
1800.0                                          0.0   0.0   0.0   0.0   0.0   
1800.1                                          0.0   0.0   0.0   0.0   0.0   
1800.2                                          0.0   0.0   0.0   0.0   0.0   
1800.3                                          0.0   0.0   0.0   0.0   0.0   
1800.4                                          0.0   0.0   0.0   0.0   0.0   
...                                             ...   ...   ...   ...   ...   
2049.6                                          0.0   0.0   0.0   0.0   0.0   
2049.7                                          0.0   0.0   0.0   0.0   0.0   
2049.8                                          0.0   0.0   0.0   0.0   0.0   
2049.9                                          0.0   0.0   0.0   0.0   0.0   
2050.0                                          0.0   0.0   0.0   0.0   0.0   

variable  ... total_populationXage_50                                \
quantile  ...                   0.025          0.250          0.500   
time      ...                                                         
1800.0    ...             5000.000000    5000.000000    5000.000000   
1800.1    ...             4996.801572    4996.801572    4996.801572   
1800.2    ...             4993.535924    4993.535924    4993.535924   
1800.3    ...             4990.204955    4990.204955    4990.204955   
1800.4    ...             4986.810554    4986.810554    4986.810554   
...       ...                     ...            ...            ...   
2049.6    ...           264633.896510  265511.854074  265997.816733   
2049.7    ...           264752.882028  265635.360005  266123.073458   
2049.8    ...           264871.693465  265758.694769  266248.156323   
2049.9    ...           264990.332638  265881.860189  266373.067171   
2050.0    ...           265108.801355  266004.858081  266497.807837   

variable                               total_populationXage_70                 \
quantile          0.750          0.975                   0.025          0.250   
time                                                                            
1800.0      5000.000000    5000.000000             5000.000000    5000.000000   
1800.1      4996.801572    4996.801572             4979.930717    4979.930717   
1800.2      4993.535924    4993.535924             4960.025764    4960.025764   
1800.3      4990.204955    4990.204955             4940.283333    4940.283333   
1800.4      4986.810554    4986.810554             4920.701644    4920.701644   
...                 ...            ...                     ...            ...   
2049.6    266407.038441  267271.175958           172685.761282  173012.478568   
2049.7    266536.286305  267403.889558           172812.591395  173137.140418   
2049.8    266665.105588  267536.427275           172938.507865  173261.541129   
2049.9    266793.638431  267668.790985     

In [10]:
target_plot = plot_output_ranges(base_quantiles,targets,['percentage_latent'],1,2010,2025)

In [11]:
target_plot

In [12]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)
plot_output_ranges(base_quantiles,targets,['mortality'],1,2010,2025)

In [13]:
target_plot = plot_output_ranges(base_quantiles,targets,['total_population'],1,2010,2025)


In [14]:
plot_output_ranges(base_quantiles,targets,['notification'],1,2010,2025)

In [28]:
plot_output_ranges(base_quantiles,targets,['acf_detectionXact3_trialXorgan_pulmonary'],1,2014,2019)

In [16]:
plot_output_ranges(base_quantiles,targets,['acf_detectionXact3_controlXorgan_pulmonary'],1,2010,2025)

In [17]:
# target_plot.write_image(DOCS_PATH / "targets2.png", scale=3)

In [18]:
# spah.write_image(DOCS_PATH / 'spah.png', scale = 3)

In [19]:
# target_plot_history = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,1800,2010, history =True)

In [20]:
# target_plot_history

In [21]:
# target_plot_history.write_image(DOCS_PATH / 'targets_history.png', scale=3)

In [22]:
# compare_target_plot = plot_output_ranges(outputs['base_scenario'],targets,['incidence','mortality_raw','prevalence_smear_positive', 'percentage_latent'],2,2010,2025)

In [23]:
# compare_target_plot.write_image(DOCS_PATH / "non_targets.png", scale='3')

In [24]:
# compare_target_plot

In [25]:
# screening_plot.write_image(DOCS_PATH / 'screening_plot.png', scale =3)

In [26]:
screening_plot

NameError: name 'screening_plot' is not defined

In [ ]:
cdr_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['case_notification_rate'],1,2010,2025)

In [ ]:
cdr_plot

In [ ]:
# cdr_plot.write_image(DOCS_PATH / 'cdr_plot.png', scale =3)

In [ ]:
early_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['incidence_early_prop'] ,1,2010,2025)

In [ ]:
early_plot

In [ ]:
# early_plot.write_image(DOCS_PATH / 'early_plot.png', scale =3)

In [ ]:
# # Simulate data extraction, replace with actual data extraction
# data_early = outputs['base_scenario']['quantiles']['incidence_early_prop'][0.5].reset_index()
# data_early['type'] = 'Early incidence'

# data_late = outputs['base_scenario']['quantiles']['incidence_late_prop'][0.5].reset_index()
# data_late['type'] = 'Late incidence'

# # Combine dataframes
# data_combined = pd.concat([data_early, data_late])
# fig = px.area(data_combined, x='time', y=0.5, color='type',
#               labels={0.5: 'Proportion (%)', 'time': ''},
#               title='',
#               range_x=[2000, 2025], range_y=[0,100])  # Update x-axis range

# # Update layout to customize legend and axis range
# fig.update_layout(
#     xaxis=dict(
#         title='',  # No x-axis title specified
#         title_font=dict(size=12)  # Setting size even if title is not shown
#     ),
#     yaxis=dict(
#         title='Incidence Proportion',
#         title_font=dict(size=12)  # Specific font size for y-axis title
#     ),
#     legend_title_text='',  # Remove legend title
#     legend=dict(
#         orientation="h",  # Horizontal legend
#         yanchor="bottom",
#         y=-0.15,  # Position the legend below the plot
#         xanchor="center",
#         x=0.5,
#         font=dict(size=12)  # Font size for legend items
#     ),
#     margin=dict(l=20, r=20, t=20, b=50),  # Adjusted margins for better spacing
#     font=dict(  # Apply font size adjustments
#         family="Arial, sans-serif",  # Set font family
#         size=12,  # Default font size for other text elements
#         color="black"  # Font color
#     )
# )

# fig.show()

In [ ]:
base_df = outputs['base_scenario']['quantiles']

# Calculate 80% of the original values for `prop_early_latent` and `prop_late_latent`


# Extract data for each compartment, label it, and store in a list
data_frames = []
for compartment in compartments:
    if f'prop_{compartment}' in base_df.columns:
        df = base_df[f'prop_{compartment}'][0.5].reset_index()  # Adjust this if your quantile structure is different
        df[0.5] *= 100  # Convert to percentage
        df['type'] = compartment.replace('_', ' ').capitalize()
        data_frames.append(df)

# Combine all compartment data into one DataFrame for plotting
combined_data = pd.concat(data_frames)

# Plot using Plotly Express
fig = px.area(combined_data, x='time', y=0.5, color='type',
              labels={'0.5': 'Proportion (%)', 'time': 'Time'},
              title='', range_x=[1980, 2025], range_y=[0, 100])

# Update layout
fig.update_layout(
    xaxis=dict(title='', title_font=dict(size=12)),
    yaxis=dict(title='<b>Proportion (%)</b>', title_font=dict(size=12), title_standoff=0),
    legend_title_text='',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    ),
    height=320,  # Set the figure height
    margin=dict(l=10, r=5, t=10, b=40),
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

# Show the plot
fig.show()


In [ ]:
# fig.write_image(DOCS_PATH / 'comps.png', scale =3)